In [1]:

import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

from sklearn import svm
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint


from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

from sklearn.utils import shuffle

import keras_tuner

from sklearn.model_selection import StratifiedKFold
import numpy as np 

from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_csv('https://media.githubusercontent.com/media/MadMoose02/COMP3608-Project/main/card_transdata.csv')
# df = pd.read_csv('card_transdata.csv')

print(f'shape of dataframe {df.shape}')

shape of dataframe (1000000, 8)


### Establishing the independent and dependent variables

In [3]:
X = pd.DataFrame(df.iloc[: , 0:7 ].values)
y = df.iloc[: ,-1].values

### Scaling the data 

In [4]:
sc = StandardScaler()
X = sc.fit_transform(X)

### Split dataset into 3 portions 

The dataset was then split into 3 different datasets used to train per model .

To effectively detect fraud, our foremost goal is to ensure an equitable distribution of positive fraud records across all three datasets.



In [5]:
fraud = df[df['fraud']== 1 ]
no_fraud = df[df['fraud'] == 0]

print(f'amount of fraud records :  {fraud.shape[0]}')
print(f'amount of Non - fraudulent records :  {no_fraud.shape[0]}')


#! shrink the dataset to match the fraud records





amount of fraud records :  87403
amount of Non - fraudulent records :  912597


In [6]:
# Down sample the data 
no_fraud = no_fraud.head(fraud.shape[0])
print(no_fraud.shape)


exclude_rows = pd.concat([fraud , no_fraud])

print(exclude_rows.shape)


# represents all the non- fraudulent transactions not used 
difference_df = df[~df.isin(exclude_rows.to_dict(orient='list')).all(axis=1)]
print(f'the difference of the dataframes{difference_df.shape}')



(87403, 8)
(174806, 8)
the difference of the dataframes(825194, 8)


In [7]:
balanced_df = pd.concat([fraud , no_fraud])
balanced_df = shuffle(balanced_df)

print(f'size in balanced df : {balanced_df.shape[0]}')


even_distr = int(balanced_df.shape[0]/3)
print( f'amount allocated to each dataset :{even_distr} ' )


df1 = balanced_df.head(even_distr)
df2 = balanced_df.iloc[(even_distr+1):(even_distr + even_distr)]
df3 = balanced_df.tail(even_distr+1)

size in balanced df : 174806
amount allocated to each dataset :58268 


In [8]:
def balance_dataset( df:pd.DataFrame , differenceDf :pd.DataFrame ) -> pd.DataFrame:
    fraud = df[df['fraud']== 0 ]
    no_fraud = df[df['fraud'] == 1]
    print(f'before balance')
    print('\n')
    print(f'fraud count is : {fraud.shape[0]} and no - fraud count is : {no_fraud.shape[0]}')


    # sample selcts data from the data frame without duplciates
    if fraud.shape[0] > no_fraud.shape[0]:
        additional_data = difference_df.sample(fraud.shape[0] - no_fraud.shape[0] , replace= False)
        no_fraud = pd.concat([no_fraud , additional_data])
        print('hi')
        print('\n')
        print(f' in fraud count : {fraud.shape[0]} and no - fraud count : {no_fraud.shape[0]}')
        merged = pd.concat([no_fraud , fraud])
        merged = shuffle(merged)
        return merged 
    elif fraud.shape[0] < no_fraud.shape[0]: 
        print('smote')
        oversample = SMOTE(random_state= 42)
        fraud_X = pd.DataFrame(df.iloc[: , 0:7 ].values)
        fraud_y = df.iloc[: ,-1].values
        fraud_X_val  , fraud_y_val  = oversample.fit_resample(fraud_X , fraud_y)

        fraud_X_val = pd.DataFrame(fraud_X_val)
        fraud_y_val = pd.DataFrame(fraud_y_val)

        mergeXandY = pd.concat([fraud_X_val,fraud_y_val] , axis = 1)

        fraud_bal = mergeXandY[mergeXandY.iloc[:, -1] == 0]
        no_fraud_bal = mergeXandY[mergeXandY.iloc[:, -1] == 1]

        print(f' fraud count {fraud_bal.shape[0]} and no - fraud count{no_fraud_bal.shape[0]}')
        print('\n')
        #merged = pd.concat([fraud_bal, no_fraud])
        merged = shuffle(mergeXandY)
        return merged 
    else:
        print('cu')
        merged = pd.concat([fraud , no_fraud])
        merged = shuffle(merged)
        return merged

    #! remember to shuffle after returned 

In [9]:
df1 = balance_dataset(df1 , difference_df)
df2 = balance_dataset(df2 , difference_df)
df3 = balance_dataset(df3 , difference_df)


before balance


fraud count is : 29233 and no - fraud count is : 29035
hi


 in fraud count : 29233 and no - fraud count : 29233
before balance


fraud count is : 29158 and no - fraud count is : 29109
hi


 in fraud count : 29158 and no - fraud count : 29158
before balance


fraud count is : 29011 and no - fraud count is : 29258
smote
 fraud count 29258 and no - fraud count29258




### Train Test Split on Dataset

In [10]:
X = pd.DataFrame(df1.iloc[: , 0:7 ].values)
y = df1.iloc[: ,-1].values

In [23]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size= 0.25)

X_train = np.array(X_train)
y_train = np.array(y_train)


Feature Scaling is optional 

### Artificial Neural Network 

Hyper Parameter tuning to determine the amount of nuerons to be used in the hidden layer 


In [24]:
def build_model(hp):

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(hp.Choice('units',[2 , 4 , 8 , 12 , 16 , 20 ,24]) , activation= 'relu') )
    model.add(tf.keras.layers.Dense(1 , activation= 'sigmoid'))
    model.compile(optimizer = 'SGD',
              loss = tf.keras.losses.MeanSquaredError(), 
              metrics = [tf.keras.metrics.Recall()])

    return model 

In [25]:
tuner = keras_tuner.RandomSearch(build_model, 
                                 objective='val_recall', 
                                 max_trials= 6)

In [26]:
tuner.search(X_train , y_train , epochs = 20 , validation_data = (X_test , y_test))

Trial 6 Complete [00h 00m 19s]
val_recall: 0.9699848294258118

Best val_recall So Far: 0.9705355763435364
Total elapsed time: 00h 01m 55s


In [11]:
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name="val_recall", direction="max")

Trial 0 summary
Hyperparameters:
units: 4
Score: 1.0

Trial 1 summary
Hyperparameters:
units: 20
Score: 1.0

Trial 2 summary
Hyperparameters:
units: 24
Score: 0.0

Trial 3 summary
Hyperparameters:
units: 12
Score: 0.0

Trial 4 summary
Hyperparameters:
units: 16
Score: 0.0


K - fold cross validation on the nueral network to determine

In [27]:
seed = 7 
np.random.seed(seed)

kfold = StratifiedKFold(n_splits=10 , shuffle= True , random_state= seed )

scores_collection = []


In [28]:
for train , test in kfold.split(X , y):
    model =  tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, activation='relu') , 
    tf.keras.layers.Dense(1 , activation= 'sigmoid')
    ])

    model.compile(optimizer = 'SGD',
              loss = tf.keras.losses.MeanSquaredError(), 
              metrics = [tf.keras.metrics.Recall()])

    model.fit(X_train,y_train , epochs= 20)
    score = model.evaluate(X_test,y_test)

    scores_collection.append(score[1] * 100 )
    print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

print("%.2f%% (+/- %.2f%%)" % (np.mean(scores_collection), np.std(scores_collection)))

Epoch 1/20
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 1s 509us/step - loss: 0.1604 - recall_2: 0.8034
Epoch 2/20
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 1s 515us/step - loss: 0.0585 - recall_2: 0.9019
Epoch 3/20
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 1s 488us/step - loss: 0.0441 - recall_2: 0.9254
Epoch 4/20
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 1s 503us/step - loss: 0.0385 - recall_2: 0.9364
Epoch 5/20
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 1s 500us/step - loss: 0.0352 - recall_2: 0.9457
Epoch 6/20
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 1s 497us/step - loss: 0.0324 - recall_2: 0.9501
Epoch 7/20
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 1s 503us/step - loss: 0.0312 - recall_2: 0.9555
Epoch 8/20
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 1s 499us/step - loss: 0.0302 - recall_2: 0.9569
Epoch 9/20
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 1s 494us/step - loss: 0.0300 - recall_2: 0.9587
Epoch 10/20
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 1s 495us/step - loss: 0.0282 - recall_2: 0.9618
Epoch 11/20
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 1s 497us/step - loss: 0.0283 - recall_2: 0.9621
Epoch 12

model =  tf.keras.models.Sequential([

    #Dense layer , number of neurons and activation
    tf.keras.layers.Dense(1 , activation='relu'),
    #output , numnber of nuerons in output layer is no.of classes 
    tf.keras.layers.Dense(1 , activation= 'sigmoid')

    #each hidden layer has one nueron 
])

loss_function = tf.keras.losses.MeanSquaredError()

model.compile(optimizer = 'SGD',
              loss = loss_function, 
              metrics = [tf.keras.metrics.Recall()])

model.fit(X_train,y_train , epochs= 5)


model.evaluate(X_test,y_test)

y_hat = model.predict(X_test)

classifying the raw neural net values
y_pred_thresholded = np.where(y_hat > 0.5, 1, 0)


conMatrix = confusion_matrix(y_test , y_pred_thresholded)
accuracyScore = accuracy_score(y_test , y_hat)

print(conMatrix)

print(f'Accuracy {accuracyScore}')


### Logistic Regression 

In [11]:
X = pd.DataFrame(df2.iloc[: , 0:7 ].values)
y = df2.iloc[: ,-1].values

In [12]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size= 0.25)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [13]:
logit = LogisticRegression(max_iter=10000 , verbose= 1)

param_grid= { 
    'C':[0.1 , 0.2 , 0.3 , 0.4 ] , 
    'penalty':['l1' , 'l2'] , 
    'solver':['liblinear' , 'saga']
    }


grid_search = GridSearchCV(estimator=logit , param_grid=param_grid , cv = 6)
grid_search.fit(X_train , y_train)


print('tuned hyper param ' ,grid_search.best_params_ )
print("accuracy :",grid_search.best_score_)
#y_hat = logit.predict(X_test)

#print(f'Accuraccy for logit is {accuracy_score(y_test,y_hat)}')


#print(f'confusion_matrix for logit is {confusion_matrix(y_test,y_hat)}')


[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 4363 epochs took 19 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 4367 epochs took 19 seconds


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 4369 epochs took 19 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 4372 epochs took 21 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   20.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [17]:
#using cardTransData1 

t_logit = LogisticRegression()

t_logit.fit(X_train , y_train)

y_nw = t_logit.predict(X_test)

print(f'{confusion_matrix(y_test , y_nw)}')
print(f'{accuracy_score(y_test , y_nw)}')

[[ 466 6792]
 [ 143 7145]]
0.5232366286264265


###  Tuned logistic regression model 


In [18]:
best_logit = LogisticRegression(C =0.3 , penalty='l1', solver = 'liblinear')

best_logit.fit(X_train , y_train)

y_nw = best_logit.predict(X_test)

print(f'{confusion_matrix(y_test , y_nw)}')

[[7121  137]
 [ 428 6860]]


### Random Forest Classifier 


In [ ]:
X = pd.DataFrame(df3.iloc[: , 0:7 ].values)
y = df3.iloc[: ,-1].values


X_train , X_test , y_train , y_test = train_test_split(X , y , test_size= 0.25)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [7]:

rf = RandomForestClassifier()

rf.fit(X_train,y_train)

y_hat = rf.predict(X_test)

print(f'Accuracy {accuracy_score(y_test,y_hat)}')

print(f'confusion matrix {confusion_matrix(y_test,y_hat)}')

KeyboardInterrupt: 

### SVM

In [ ]:

scaler = StandardScaler() 

X_train = scaler.fit_transform(X_train)

clf = LinearSVC(max_iter=200000, dual = False)



clf.fit(X_train,y_train)
print('y')

y_hat = clf.predict(X_test)
print('t')

y
t


In [ ]:
print(f'Accuracy for SVM is {accuracy_score(y_test,y_hat)}')





Accuraccy for SVM is 0.234472
